In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction/Output')
cur_date = "080123"

library(readxl)
library(openxlsx)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)
library(xgboost)
library(rlang)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_051023.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.2     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.2     ✔ tibble  3.2.1
✔ purrr   1.0.1     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
#StandWithUkraine

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Ar,1.438029,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Ar,1.863054,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Ar,1.434020,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Ar,35.000000,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Ar,1.470804,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Ar,53.000000,D


Using `water_sample_date`, `casing_depth`, `well_depth`, `static_water_depth`, `pH`, and `flow_rate` to predict As concentration. RF models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. < or > 5ppb
3. < or > 10ppb

Starting by creating 2 additional variables for above and below 5 and 10ppb and calculating some summary statistics to determine if there are any signficiant differenes between the predictor variables for each outcome variable.

In [3]:
arsenic_data = well_data %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration >= 5, 1, 0)), ref = "0"),
          Ten_Threshold = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
           # making this col into a factor
          Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date),
          # couldn't leave this as a date type, since xgboost only understands numerical data therefore the 
          # duration between 1st sample date and each date will be used to make a numerical col
          Water_Sample_Numeric = as.numeric(Water_Sample_Date - min(na.omit(Water_Sample_Date)))) %>% #,
          # creating a new col where the sample date is categorized into seasons
#           Sample_Date_Season = ifelse(month(Water_Sample_Date) < 6 & month(Water_Sample_Date) > 2, "Sp", # spring
#                          ifelse(month(Water_Sample_Date) < 9 & month(Water_Sample_Date) > 5, "Su", # summer
#                          ifelse(month(Water_Sample_Date) < 12 & month(Water_Sample_Date) > 8, "Fa", # fall
#                                 "Wi")))) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar")

# rearranging cols
arsenic_data = arsenic_data[,c(1:4,15,5:14)]
head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  9 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Water_Sample_Numeric,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<fct>
1,1006004,960,12-155,2012-09-24,1560,52,165,41,60.0,7.7,Ar,1.438029,0,0,0
2,1024009,1803,15-177,2015-12-17,2739,40,445,42,2.0,7.3,Ar,1.863054,0,0,0
3,1054019,1647,14-212,2015-02-02,2421,45,160,40,40.0,7.4,Ar,1.434020,0,0,0
4,1057017,970,12-169,2012-10-22,1588,42,440,57,1.5,8.0,Ar,35.000000,1,1,1
5,1060006,533,10-239,2011-01-03,930,48,120,42,25.0,7.1,Ar,1.470804,0,0,0
6,1066006,1795,15-224,2015-12-15,2737,60,280,32,10.0,8.2,Ar,53.000000,1,1,1


In [4]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(4:10,13)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 731","**1**, N = 292",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-12 (942.199556816559),2012-12-08 (983.231689071497),0.019
Water_Sample_Numeric,"1,020","1,791 (942)","1,636 (983)",0.019
Casing_Depth,"1,023",73 (33),56 (23),<0.001
Well_Depth,"1,023",306 (148),340 (137),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.066
Flow_Rate,"1,023",25 (34),14 (17),<0.001
pH,"1,023",7.41 (0.55),7.78 (0.39),<0.001


In [5]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:10,14)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 735","**1**, N = 288",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-04 (946.639732018247),2012-12-28 (975.861944951949),0.057
Water_Sample_Numeric,"1,020","1,782 (947)","1,655 (976)",0.057
Casing_Depth,"1,023",73 (33),56 (23),<0.001
Well_Depth,"1,023",305 (148),341 (137),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.084
Flow_Rate,"1,023",25 (33),14 (17),<0.001
pH,"1,023",7.41 (0.55),7.79 (0.39),<0.001


In [6]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:10,15)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 819","**1**, N = 204",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-09 (948.708869245644),2012-10-14 (970.291642552518),0.005
Water_Sample_Numeric,"1,020","1,788 (949)","1,580 (970)",0.005
Casing_Depth,"1,023",72 (33),53 (20),<0.001
Well_Depth,"1,023",310 (150),339 (127),0.010
Static_Water_Depth,"1,023",35 (14),37 (16),0.041
Flow_Rate,"1,023",24 (32),13 (16),<0.001
pH,"1,023",7.45 (0.55),7.80 (0.39),<0.001


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance. Class imbalance will be addressed using SMOTE. We can also see water sample date and water sample numeric are yielding the same p values, which is great. 

In [7]:
# creating dfs for each outcome
# dropped 3 rows that had missing dates
arsenic_detect_df = drop_na(arsenic_data[,c(5:10,13)]) 
arsenic_five_df = drop_na(arsenic_data[,c(5:10,14)]) 
arsenic_ten_df = drop_na(arsenic_data[,c(5:10,15)]) 

head(arsenic_detect_df)

,Water_Sample_Numeric,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1560,52,165,41,60.0,7.7,0
2,2739,40,445,42,2.0,7.3,0
3,2421,45,160,40,40.0,7.4,0
4,1588,42,440,57,1.5,8.0,1
5,930,48,120,42,25.0,7.1,0
6,2737,60,280,32,10.0,8.2,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [14]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df, "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [15]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3), Kernel = NA,
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]]))

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,NA,Detect_Concentration,0.6978623,0.5825833,0.8131412,0.5557035,0.6978623
RF Classification,NA,Five_Threshold,0.7067813,0.6007864,0.8127761,0.5607455,0.7067813
RF Classification,NA,Ten_Threshold,0.7002233,0.5389024,0.8615442,0.4942215,0.7002233


Predictor,MeanDecreaseGini
<chr>,<dbl>
pH,140.85426
Casing_Depth,134.07336
Flow_Rate,97.75841
Static_Water_Depth,88.01873
Well_Depth,87.84037
Sample_Date_Season,29.44228


Predictor,MeanDecreaseGini
<chr>,<dbl>
pH,141.23835
Casing_Depth,132.96978
Flow_Rate,101.21615
Well_Depth,88.85529
Static_Water_Depth,87.75080
Sample_Date_Season,29.21067


Predictor,MeanDecreaseGini
<chr>,<dbl>
Casing_Depth,153.25072
pH,147.36309
Flow_Rate,109.27892
Static_Water_Depth,106.45938
Well_Depth,97.92386
Sample_Date_Season,33.42404


RF didn't perform as well as I hoped, so let's try another model - SVM. 

# Support Vector Machine (SVM)
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [16]:
# model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        
        data_test = dataset[dataset_index[[i]],]

        # now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        # choosing best model
        best_svm <- svr_tune$best.model

        # predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  # taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [17]:
# calling fn
# linear
linear_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_five = svm_classification(arsenic_five_df, "linear", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "linear", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# radial
radial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "radial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "radial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "radial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# polynomial
polynomial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "polynomial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "polynomial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "polynomial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [18]:
# viewing results
svm_confusion_matrix = data.frame(Model = "SVM Classification",
        Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
        Kernel = rep(c(rep(c("linear"), times = 3), rep(c("radial"), times = 3), rep(c("polynomial"), times = 3))),
            rbind(linear_svm_values_arsenic_detect, linear_svm_values_arsenic_five, linear_svm_values_arsenic_ten, 
            radial_svm_values_arsenic_detect, radial_svm_values_arsenic_five, radial_svm_values_arsenic_ten, 
            polynomial_svm_values_arsenic_detect, polynomial_svm_values_arsenic_five, 
            polynomial_svm_values_arsenic_ten))

svm_confusion_matrix

Model,Outcome,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Detect_Concentration,linear,0.7164004,0.7570427,0.6757581,0.4838209,0.7164004
SVM Classification,Five_Threshold,linear,0.7225769,0.7744707,0.6706831,0.4823485,0.7225769
SVM Classification,Ten_Threshold,linear,0.7198665,0.7889024,0.6508305,0.3633425,0.7198665
SVM Classification,Detect_Concentration,radial,0.7082217,0.7572180,0.6592253,0.4729153,0.7082217
SVM Classification,Five_Threshold,radial,0.7143065,0.7743497,0.6542633,0.4703764,0.7143065
SVM Classification,Ten_Threshold,radial,0.7083061,0.7351220,0.6814903,0.3710243,0.7083061
SVM Classification,Detect_Concentration,polynomial,0.6207522,0.9351841,0.3063203,0.3507425,0.6207522
SVM Classification,Five_Threshold,polynomial,0.6229071,0.9235330,0.3222812,0.3496063,0.6229071
SVM Classification,Ten_Threshold,polynomial,0.6299955,0.9265854,0.3334057,0.2591534,0.6299955


## SVM - Variable Importance
Unfortunately, I was unable to find a function that extracts variable importance as with random forest. However, variable importance is simply the difference in overall predictivity on a model in that variable's absence in the model. Therefore, SVM models will be rerun without one variables each time. Additionally, negative value for variable importance for SVM means that inclusion of that predictor in models hurt predictivity while a positive number means that inclusion of that predictor improved predictivity. Since linear SVM models performed the best only those variable importance values will be extracted.

Getting variable importance for ....

In [19]:
# model errors are calculated to assess the predictive accuracy of the model
svm_var_imp = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    metrics = data.frame()
    # iterating through variables in the dataset
    predictors = unique(colnames(dataset[-7]))
    for (i in 1:length(predictors)){
        subsetted_df = dataset[,colnames(dataset[-i])]
        
        # splitting data into training and testing sets
        dataset_index = createFolds(subsetted_df[[outcome]], k = 5) #5 fold CV
        for (j in 1:length(dataset_index)){
            data_train = subsetted_df[-dataset_index[[j]],]
            # using SMOTE to balance the class distribution
            balanced_data_train = smotenc(data_train, outcome)

            data_test = subsetted_df[dataset_index[[j]],]

            # now pruning parameters (based on the training dataset to prevent overfitting)
            svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                           ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

            # choosing best model
            best_svm <- svr_tune$best.model

            # predicting with tuned parameters 
            data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

            matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

            # extracting accuracy to take mean later
            matrix_values = data.frame(t(c(matrix$byClass[11])), predictors[i])

            # adding values to df
            metrics = rbind(metrics, matrix_values)
        }
     }
    
    colnames(metrics) = c("Balanced Accuracy", "Predictor")

    # taking averages/sd 
     metrics = metrics %>%
        group_by(Predictor) %>%
        summarise(`Balanced Accuracy` = mean(`Balanced Accuracy`)) %>%
        mutate(Outcome = outcome)

    return(metrics)
}

In [20]:
# calling fn
# linear
linear_svm_var_imp_arsenic_detect = svm_var_imp(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
linear_svm_var_imp_arsenic_five = svm_var_imp(arsenic_five_df, "linear", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
linear_svm_var_imp_arsenic_ten = svm_var_imp(arsenic_ten_df, "linear", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# radial
radial_svm_var_imp_arsenic_detect = svm_var_imp(arsenic_detect_df, "radial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
radial_svm_var_imp_arsenic_five = svm_var_imp(arsenic_five_df, "radial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
radial_svm_var_imp_arsenic_ten = svm_var_imp(arsenic_ten_df, "radial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

In [21]:
# creating 1 var imp df
#svm
svm_var_imp_df = rbind(linear_svm_var_imp_arsenic_detect, linear_svm_var_imp_arsenic_five, 
                    linear_svm_var_imp_arsenic_ten, radial_svm_var_imp_arsenic_detect, 
                       radial_svm_var_imp_arsenic_five, radial_svm_var_imp_arsenic_ten) %>%
    # var importance value for svm = balanced accuracy with a predictor - balanced accuracy w/o a predictor
    mutate(Importance = svm_confusion_matrix$Balanced.Accuracy - `Balanced Accuracy`, 
    # adding a col for the model
           Model = c(rep(c("Linear-SVM"), times = 18), rep(c("Radial-SVM"), times = 18))) %>% 
    select(-`Balanced Accuracy`)
#rf
rf_var_imp_df = data.frame(rbind(rf_values_arsenic_detect[[3]], rf_values_arsenic_five[[3]], 
                                 rf_values_arsenic_ten[[3]]),
                Outcome = c(rep(c("Detect_Concentration"), times = 6), rep(c("Five_Threshold"), times = 6), 
                           rep(c("Ten_Threshold"), times = 6))) %>%
    # renaming mean decrease gini to match the importance value for SVM
    rename(Importance = MeanDecreaseGini) %>%
    # adding a col for the model
    mutate(Model = "RF")

var_imp_df = rbind(svm_var_imp_df, rf_var_imp_df)

head(var_imp_df)

Predictor,Outcome,Importance,Model
<chr>,<chr>,<dbl>,<chr>
Casing_Depth,Detect_Concentration,0.028882253,Linear-SVM
Flow_Rate,Detect_Concentration,0.016693418,Linear-SVM
Sample_Date_Season,Detect_Concentration,0.003447151,Linear-SVM
Static_Water_Depth,Detect_Concentration,-0.006515136,Linear-SVM
Well_Depth,Detect_Concentration,0.004931633,Linear-SVM
pH,Detect_Concentration,0.033865744,Linear-SVM


# XGBoost
- short for Extreme Gradient Boosting, XGBoost has both linear and tree learning algorithms...

In [47]:
# model errors are calculated to assess the predictive accuracy of the model
xgboost_classification = function(dataset, outcome, pred_outcome){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    nrounds_values = c(50, 250, 500) # number of trees 

    
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        # converting the outcome from a factor to numeric for xgboost
        balanced_data_train[[outcome]] = as.numeric(balanced_data_train[[outcome]]) - 1
        data_test[[outcome]] = as.numeric(data_test[[outcome]]) - 1
        # converting training and testing datasets into matrices, which xgboost prefers
        matrix_data_train = xgb.DMatrix(data = as.matrix(balanced_data_train[,1:6]), 
                                label = balanced_data_train[[outcome]])
        matrix_data_test = xgb.DMatrix(data = as.matrix(data_test[,1:6]), label = data_test[[outcome]])
        
        for (j in 1:length(nrounds_values)){

            # now pruning parameters (based on the training dataset to prevent overfitting)
            reg_xgboost <- xgboost(data = matrix_data_train, nrounds = nrounds_values[j], verbose = 0, 
                                   max.depth = 2, objective = "binary:logistic") 


            # predicting with tuned parameters at a 50% threshold
            data_test[[pred_outcome]] <- predict(reg_xgboost, newdata = matrix_data_test) 
            data_test = data_test %>%
                mutate(place_holder = relevel(factor(ifelse(eval(rlang::parse_expr(pred_outcome)) < 0.5 , 
                                                                         "0", "1")), ref = "1")) %>%
                select(-all_of(pred_outcome))
            # had to do weird things to rename the col since I couldn't pass an argument as a variable name
            colnames(data_test)[8] = pred_outcome

#             matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
#                                      positive = "1")

#             # calculating AUC
#             auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], 
#                                                                           ordered = TRUE))
#             # extracting accuracy, sens, spec, PPV to take mean later
#             matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)

#             # extracting variable importance
#             var_importance_values = data.frame(xgb.importance(reg_xgboost)) %>%
#                 rownames_to_column(var = "Predictor")
#             variable_importance_df = rbind(variable_importance_df, var_importance_values)

#             # adding values to df
#             metrics = rbind(metrics, matrix_values)
        }
  }
  
  # taking averages/sd 
#   metrics = metrics %>%
#     summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
#               Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
  # return training set, matrix, variable importance values
  #return(list(balanced_data_train, metrics, variable_importance_df))
  return(data_test)

}

# calling fn
a = xgboost_classification(arsenic_detect_df, "Detect_Concentration", "pred_Detect_Concentration")
head(a)

,Water_Sample_Numeric,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration,pred_Detect_Concentration
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
8,203,60,160,40,25,7.2,1,0
29,2729,39,340,30,4,7.4,0,1
30,644,50,200,20,25,7.6,1,0
31,790,96,250,30,15,7.7,1,0
37,3403,41,300,59,12,7.9,1,0
39,2926,60,300,40,12,8.0,0,1


In [22]:
# combining in 1 df
final_df = full_join(rf_confusion_matrix, svm_confusion_matrix) %>%
    arrange(Outcome, -Balanced.Accuracy)

final_df

Joining with `by = join_by(Model, Kernel, Outcome, Balanced.Accuracy,
Sensitivity, Specificity, PPV, AUC)`


Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,linear,Detect_Concentration,0.7164004,0.7570427,0.6757581,0.4838209,0.7164004
SVM Classification,radial,Detect_Concentration,0.7082217,0.7572180,0.6592253,0.4729153,0.7082217
RF Classification,NA,Detect_Concentration,0.6978623,0.5825833,0.8131412,0.5557035,0.6978623
SVM Classification,polynomial,Detect_Concentration,0.6207522,0.9351841,0.3063203,0.3507425,0.6207522
SVM Classification,linear,Five_Threshold,0.7225769,0.7744707,0.6706831,0.4823485,0.7225769
SVM Classification,radial,Five_Threshold,0.7143065,0.7743497,0.6542633,0.4703764,0.7143065
RF Classification,NA,Five_Threshold,0.7067813,0.6007864,0.8127761,0.5607455,0.7067813
SVM Classification,polynomial,Five_Threshold,0.6229071,0.9235330,0.3222812,0.3496063,0.6229071
SVM Classification,linear,Ten_Threshold,0.7198665,0.7889024,0.6508305,0.3633425,0.7198665


In [20]:
# # exporting
# write.xlsx(final_df, paste0(Output,"/", "As_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), rowNames = FALSE)
# write.xlsx(var_imp_df, paste0(Output,"/", "Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)